In [1]:
import importlib
import sys
import tqdm
from src.orca_hls_utils.s3_utils import get_all_folders, verify_playlist
import pandas as pd


In [18]:

# Reload the module to ensure the latest version is used
# importlib.reload(sys.modules['src.orca_hls_utils.s3_utils'])

# Define the S3 bucket and the prefix for the hydrophone (e.g. rpi_orcasound_lab)
BUCKET = "audio-orcasound-net"
PREFIX = "rpi_orcasound_lab/hls/"

# Get all timestamp folders using s3_utils.get_all_folders
folders = get_all_folders(BUCKET, PREFIX)
print(f"Found {len(folders)} folders under prefix: {PREFIX}")
df_time = pd.DataFrame(folders, columns=['folder_name'])
df_time['timestamp'] = pd.to_datetime(df_time['folder_name'], unit='s')
# Calculate the difference in seconds between consecutive timestamps
df_time['time_diff_seconds'] = df_time['timestamp'].diff().dt.total_seconds()
display(df_time)

Found 16247 folders under prefix: rpi_orcasound_lab/hls/


/tmp/ipykernel_4363/406226475.py:12: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df_time['timestamp'] = pd.to_datetime(df_time['folder_name'], unit='s')


,folder_name,timestamp,time_diff_seconds
0,1541027406,2018-10-31 23:10:06,NaN
1,1541028025,2018-10-31 23:20:25,619.0
2,1541032337,2018-11-01 00:32:17,4312.0
3,1541039542,2018-11-01 02:32:22,7205.0
4,1541046734,2018-11-01 04:32:14,7192.0
...,...,...,...
16242,1745823620,2025-04-28 07:00:20,86401.0
16243,1745910019,2025-04-29 07:00:19,86399.0
16244,1745996419,2025-04-30 07:00:19,86400.0
16245,1746082818,2025-05-01 07:00:18,86399.0


In [29]:
filtered_df_time = df_time[(df_time['timestamp'] >= '2023-11-11') & (df_time['timestamp'] <= '2024-11-11')]
display(filtered_df_time)


,folder_name,timestamp,time_diff_seconds
14811,1699669819,2023-11-11 02:30:19,21601.0
14812,1699691418,2023-11-11 08:30:18,21599.0
14813,1699694485,2023-11-11 09:21:25,3067.0
14814,1699713019,2023-11-11 14:30:19,18534.0
14815,1699734618,2023-11-11 20:30:18,21599.0
...,...,...,...
16035,1731184219,2024-11-09 20:30:19,11449.0
16036,1731205818,2024-11-10 02:30:18,21599.0
16037,1731227418,2024-11-10 08:30:18,21600.0
16038,1731249019,2024-11-10 14:30:19,21601.0


In [ ]:
report = []
if not folders:
    print(f"No folders found under prefix: {PREFIX}")
else:
    for folder in tqdm.tqdm(folders[16040:16119]):  
        folder_prefix = PREFIX + folder + "/"  # e.g. "rpi_orcasound_lab/hls/1541061134/"
        res = verify_playlist(BUCKET, folder_prefix)
        if res:
            report.append({'folder': folder} | res)

  0%|          | 0/79 [00:00<?, ?it/s]

 13%|█▎        | 10/79 [00:13<01:36,  1.40s/it]

In [24]:
import pandas as pd
df = pd.DataFrame(report)
df['timeStamp'] = pd.to_datetime(df['folder'], unit='s')
df['extra_files'] = df['extra_files'].apply(lambda x: sorted(x))
df['len_extra'] = df['extra_files'].apply(lambda x: len(x))
df['len_missing'] = df['missing_files'].apply(lambda x: len(x))
# Calculate the difference in seconds between consecutive timestamps
df['time_diff_seconds'] = df['timeStamp'].diff().dt.total_seconds()

df.to_csv("playlist_report.csv", index=False)

/tmp/ipykernel_4363/4161519107.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timeStamp'] = pd.to_datetime(df['folder'], unit='s')


In [25]:
display(df)

,folder,missing_files,extra_files,timeStamp,len_extra,len_missing,time_diff_seconds
0,1732264230,[2157],[],2024-11-22 08:30:30,0,1,NaN
1,1732804230,[2157],[],2024-11-28 14:30:30,0,1,540000.0


In [52]:
df['extra_files'].iloc[0]

['351', '353', '355', '356', '358', '352', '354', '357']

In [57]:
df['len_extra'].value_counts()

len_extra
0    569
7      3
8      1
Name: count, dtype: int64

In [52]:
df['extra_files'].iloc[0]

['351', '353', '355', '356', '358', '352', '354', '357']